# Relazione terzo progetto algoritmi avanzati

Francesca Meneghello 1227939 <br />
Leonardo Pratesi 1237582

## 1. Introduzione

Spiegazione della struttura utilizzata

In [1]:
import Contraction as c

lista, nodi= c.ListEdge('input_random_1_6.txt')
print(lista)

[('3', '5'), ('1', '4'), ('1', '2'), ('1', '3'), ('2', '5'), ('2', '4'), ('5', '6'), ('4', '5'), ('2', '6')]


## 2. Il tempo impiegato dalla procedura di Full Contraction


Spiegazione grafico, media del tempo impiegato dalla FullContraction per dimensione grafo 

<img src="FullContractionMedia.png" width=550 height=450 />

## 3. Il tempo impiegato dall'algoritmo completo per ripetere la contrazione un numero sufficientemente alto di volte

## 4. Il discovery time, ossia il momento in cui l'algoritmo trova per la prima volta il taglio di costo mimimo

## 5. L'errore nella soluzione trovata rispetto al risultato ottimo

## 6. Conclusioni